# Objectif

- Prédire les émissions de CO2 (TotalGHGEmissions)
- Prédire la consommation totale d'énérgie (SiteEnergyUseWN(kBtu) ou SiteEnergyUse(kBtu))
- Prendre en compte uniquement les bâtiments non destinés à l'habitation
- Evaluer l'intérêt de la feature "ENERGYSTARScore" car fastidieux à calculer

L'énoncé nous précise que ces données sont compliquées à obtenir d'où l'intérêt de créer un modèle d'IA pouvant prédire les données futurs à partir d'un échantillon de 2016

## Targets

**Nos targets sont** :

- TotalGHGEmissions
- SiteEnergyUse(kBtu) ou SiteEnergyUseWN(kBtu)

## Itérations d'analyse / nettoyage

Itération 1 : shape (1331, 22)

Itération 2 : shape ()

<hr />

# Exploratory Data Analysis


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
np.set_printoptions(linewidth=180)
sns.set_style("dark")
sns.axes_style("darkgrid")

%matplotlib inline

# Servira pour la partie analyse
target_cols = ["TotalGHGEmissions", "SiteEnergyUse(kBtu)"]

In [ ]:
data = pd.read_csv("2016_Building_Energy_Benchmarking.csv")
df = data.copy()
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

Suppression des données qui ne seront pas accessibles lors de l'utilisation du modèle après entrainenement et qui ne sont pas nos targets (éviter le data leakage)


In [ ]:
df = df.drop(
    columns=[
        "SiteEUI(kBtu/sf)",
        "SiteEUIWN(kBtu/sf)",
        "SourceEUI(kBtu/sf)",
        "SourceEUIWN(kBtu/sf)",
        "SteamUse(kBtu)",
        "Electricity(kWh)",
        "Electricity(kBtu)",
        "NaturalGas(therms)",
        "NaturalGas(kBtu)",
        "SiteEnergyUseWN(kBtu)",
        "GHGEmissionsIntensity"
    ]
)

On regarde les colonnes et leur type


In [ ]:
df.dtypes

In [ ]:
df.nunique().sort_values()

In [ ]:
df.nunique().sort_values()

In [ ]:
df[df.nunique().index[df.dtypes == "object"]].nunique().sort_values()

Pour rappel, le nombre de lignes que nous avons est de 3376.

D'après les données catégorielles ci-dessus, on peut dire qu'il y a peu de variances pour celles en bas. Ainsi on peut les retirer les données catégorielles avec trop ou peu de variance pour filtrer plus rapidement nos features.

On peut partir du principe que si la variance est plus de 90%, alors la feature ne sera pas exploitable.
A l'inverse, si la seule donnée dans la feature que nous avons est unique, alors on peut aussi la supprimer

In [ ]:
for column in df.columns[df.dtypes == "object"]:
    num_uniques = df[column].unique().size
    variance = num_uniques / df[column].value_counts().sum()
    if(variance > 0.9 or num_uniques <= 1):
        df = df.drop(columns=column)
    
df.shape

On passe de 46 à 31 features !

In [ ]:
df.dtypes.value_counts().plot.pie(autopct='%.2f %%')

On a affaire à un dataset avec des données de types variés. Le type prédominant est quand même le nombre flotant, ainsi on aura certainement par la suite des analyses continues à effectuer.

On voit également qu'on a beaucoup de features. On peut les traiter une par une pour comprendre leur possible impacte lors de nos prédictions.

## Nulls

On peut se faire une petite fonction pour examiner le nombre de nulls de notre dataset. On pourra l'utiliser pour voir si nos traitements

In [ ]:
def nulls(df):
    plt.figure(figsize=(20, 8))
    plt.title("Répartition des valeurs non définies sous forme de heatmap")
    sns.heatmap(df.isna(), cbar=False)
    plt.xticks(rotation=45, horizontalalignment="right")
    plt.plot()


nulls(df)

On remarque qu'il a beaucoup de features pour lesquelles les données sont toutes définies.

Il y a seulement quelques données pour les features : LargestPropertyUseTypeGFA ; TotalGHGEmissions, NumberofBuildings... dont les données sont manquantes.

In [ ]:
(df.isna().sum() / df.shape[0]).sort_values(ascending=False)

On remarque que certaines features possèdent beaucoup de valeurs manquantes. A voir si ces features serait utiles à la prédiction.

On a également quelques lignes impactées par un manque de valeur. Ce qui est intéressant est que les valeurs manquantes sont réparties en lignes pour ces données. Ainsi on pourra certainement simplement supprimer ces quelques lignes pour gagner du temps et éviter de faire des traitements inutiles.

On peut commencer par supprimer la colonne "Comments" qui ne contient que des nulls.

In [ ]:
df = df.drop(columns="Comments")

In [ ]:
features = df.columns.to_list()

def next_feature():
    if(len(features) == 0): return None
    feature = features.pop(0)
    return feature

On peut stoquer nos colonnes dans un tableau et itérer sur chaque variable pour les analyser une par une.

Eventuellement on pourra analyser des features entre elles.

In [ ]:
next_feature()

## OSEBuildingID

In [ ]:
df.OSEBuildingID.unique().size

In [ ]:
df.OSEBuildingID.max()

Même taille que le nombre de lignes. Mais le l'id max est bien plus élevé à l'id max (3376). Ce dataset est possiblement un échantillon d'un dataset plus large avec au moins 50226 lignes.

In [ ]:
df = df.drop(columns="OSEBuildingID")

On peut retirer la feature OSEBuildingID étant donné que toutes les valeurs sont différentes. Elle ne seront d'aucune utilité pour effectuer nos prédictions

In [ ]:
next_feature()

In [ ]:
df.DataYear.unique()

DataYear est une feature représentant l'année du de prélévement des données. On peut s'en séparer immédiatement. Analysons plutôt l'année de construction des bâtiment dès le début ce qui nous sera utile pour les comparaisons.

In [ ]:
features.remove("YearBuilt")
df.drop(columns="DataYear", inplace=True)

## YearBuilt

YearBuilt est une feature intéressante. On pourra l'utiliser en temps voulu avec d'autres features

In [ ]:
df.YearBuilt.isna().value_counts()

In [ ]:
unique_years = np.sort(df.YearBuilt.unique())
unique_years

In [ ]:
df.YearBuilt.describe()

On a beaucoup de valeurs différentes pour ce qui est de l'année de construction des bâtiments. Pour nos analyses et prédiction il serait bien d'avoir des valeurs plus catégorielles

In [ ]:
year_range_min = min(unique_years)
year_range_max = max(unique_years)
year_range_min_10 = int(int(year_range_min / 10) * 10)
year_range_max_10 = int(int(year_range_max / 10) * 10)
year_range_bins = range(year_range_min_10, year_range_max_10 + 20, 10)
year_range_labels = [
    "{}-{}".format(x, x + 9) for x in range(year_range_min_10, year_range_max_10, 10)
]
year_range_labels.append("2010+")

df["YearBuiltRange"] = pd.cut(
    df.YearBuilt, bins=year_range_bins, labels=year_range_labels, include_lowest=True
)


df["YearBuiltRange"].value_counts().sort_index().plot(kind="bar")
plt.title("Fréquence des années")
plt.xlabel("Intervalles d'années")
plt.ylabel("Fréquence")
df[["YearBuilt", "YearBuiltRange"]].head()

- On remarque un creux vers les années 1930-1950 qui pourrait s'expliquer par la présence de la seconde guerre mondiale
- Peu de données pour 2010+ étant données que 2016 correspond à 3/5 d'une intervalle de 10 ans.

In [ ]:
next_feature()

## BuildingType

In [ ]:
df.BuildingType.isna().value_counts()

In [ ]:
df.BuildingType.unique()

In [ ]:
df.BuildingType.value_counts().plot.pie()

Malgré le fait que les bâtiments destinés à l'habitation représentent 50% de notre jeu de données, comme on souhaite effectuer des prédictions uniquement sur les bâtiment destinées à autre chose que l'habitation, on va devoir s'en séparer.

In [ ]:
df = df[~df.BuildingType.isin(["Multifamily MR (5-9)", "Multifamily LR (1-4)", "Multifamily HR (10+)"])]
df.BuildingType.unique()

Nonresidential COS :
- _"Compulsory Open Space (COS) means minimum part of a plot which is to be left completely and compulsorily open to sky under the regulations"_.

    Il peut être utile de conserver cette données telle quelle car elle pourrait caractériser des données différement du simple "NonResidential"

SPS-District K-12
- _"est couramment employé aux États-Unis pour désigner l'ensemble du cursus scolaire, de la maternelle (Kindergarten) au secondaire (grade 12)"_

    On peut se demander quelle différence avec _"Campus"_. Mais un campus peut aussi bien être un lieux d'étude qu'un lieu de travail ordinaire

Nonresidential WA:
- WA signifiant WAshington, on peut remplacer cette donnée par NonResidential

In [ ]:
df.BuildingType.replace("Nonresidential WA", "NonResidential", inplace=True)
df.BuildingType.unique()

In [ ]:
df.BuildingType.value_counts().plot.pie(autopct='%.2f %%')

On a 88% de NonResidential. Ceci signifie qu'il y a peu de variance avec cette feature. On pourra voir par la suite si on conserve cette feature.

### BuildingType - YearBuiltRange

On peut d'ores et déjà comparer le type de bâtiment à l'année pour y voir un lien.

In [ ]:
cross = pd.crosstab(df.BuildingType, df.YearBuiltRange, margins=True)
cross

- Aucun campus n'a encore été créé en 2010+
- Il n'y avait pas de SPS-District K-12 avant 1920

In [ ]:
next_feature()

## PrimaryPropertyType

In [ ]:
df.PrimaryPropertyType.isna().value_counts()

In [ ]:
df.PrimaryPropertyType.unique()

On doit retirer tout ce qui concerne une résidence

In [ ]:
df = df[
    ~df.PrimaryPropertyType.isin(
        values=["Hotel", "Mixed Use Property", "Residence Hall", "Low-Rise Multifamily"]
    )
]
df.PrimaryPropertyType.unique()

In [ ]:
sns.countplot(x=df.PrimaryPropertyType)
plt.xticks(rotation=45, horizontalalignment="right");

- La catégorie "Autre", Small/Mid-Sized Office, Warehouse et Large Office sont prédominants 

### PrimaryPropertyType - BuildingType

In [ ]:
cross = pd.crosstab(df.BuildingType, df.PrimaryPropertyType, margins=True)
cross

- Les **hopitaux**, **laboratoires**, **"Refrigerated Warehouse"**, **"Residence Hall"**, **"Retail Store"**, **"Self-Storage Facility"**, **"Senior Care Community"**, **"Supermarket / Grocery Store"**, **"Worship Facility"**  sont des bâtiments non-résidentiels uniquement
- Les bureaux sont des "Nonresidential COS"
- K-12 School n'est pas forcément lié au SPS-District K-12 contrairement à ce qu'on pourrait croire

In [ ]:
next_feature()

## ZipCode

In [ ]:
df.ZipCode.isna().value_counts()

On sait que dans nos données nous avons accès à la latitude et longitude pour récupérer la position du bâtiment dans Seattle.

On peut très certainement s'appuyer sur ces données pour corriger le ZipCode.

In [ ]:
df.Latitude.isna().sum() + df.Longitude.isna().sum()

Waw ! On a 0 nulls pour nos coordonnées géographiques. On peut donc directement essayer de retrouver le ZipCode à partir de ces données.

Mais avant, retirons latitude et longitude de notre tableau de features à analyser. On utilisera ces features lorsque le besoin se ressentira pour comparer à une autre feature.

In [ ]:
features.remove("Longitude")
features.remove("Latitude")

D'après le format du ZipCode, on est sur un problème de classification. On va utiliser un algorithme de voisins pour retrouver à quelle ZipCode appartient nos 12 lignes manquantes.

In [ ]:
df_zip = df.loc[~df.ZipCode.isna(), ["Latitude", "Longitude", "ZipCode"]]

In [ ]:
plt.figure(figsize=(16,14))
g = sns.scatterplot(data=df_zip, x="Longitude", y="Latitude", hue="ZipCode", s=25, palette="Set1")
g.set_title("Clusters of the ZipCode by Latitude/Longitude")
plt.xticks(rotation=45, horizontalalignment="right");

On remarque bien certains clusters

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

X = df_zip.iloc[:,:-1]
y = df_zip.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

model = KNeighborsClassifier(n_neighbors=5)

model.fit(X_train, y_train)
model.score(X_test, y_test)

Peut-on améliorer les performances ?

In [ ]:
from sklearn.model_selection import validation_curve

k = np.arange(1, 15)

train_score, val_score = validation_curve(model, X_train, y_train, param_name='n_neighbors', param_range=k, cv=5)

plt.plot(k, val_score.mean(axis=1))
plt.title("n_neighbors - val_score")

k_best = k[val_score.mean(axis=1).argmax()]
k_best

In [ ]:
model = KNeighborsClassifier(n_neighbors=k_best)
model.fit(X_train, y_train)
model.score(X_test, y_test)

On peut donc faire nos prédictions avec ce modèle

In [ ]:
df_zip = df.loc[df.ZipCode.isna(), ["Latitude", "Longitude", "ZipCode"]]

X = df_zip.iloc[:,:-1]
y_pred = model.predict(X)
df.loc[df.ZipCode.isna(), "ZipCode"] = y_pred
df.ZipCode.isna().value_counts()

On peut également transformer les flotants en entiers car la virgule ne nous sert à rien.

In [ ]:
df["ZipCode"] = df.ZipCode.astype("int32")

### YearBuiltRange - ZipCode

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(data=df, x="YearBuiltRange", y="ZipCode")
plt.xticks(rotation=45, horizontalalignment="right");

Pour les ZipCode inférieurs à ~98100 il n'y a que très peu de ZipCode par années de construction

In [ ]:
df[df.ZipCode < 98100].shape[0]

In [ ]:
df[df.ZipCode < 98100]

Rien d'anormal en regardant les features associées aux outliers bas du ZipCode.

On va quand même conserver ces outliers si jamais ils contiennent d'autres features dont notre modèle pourrait avoir besoin.

In [ ]:
next_feature()

## CouncilDistrictCode

In [ ]:
df.CouncilDistrictCode.isna().value_counts()

In [ ]:
df.CouncilDistrictCode.value_counts().plot(kind="bar")
plt.title("Number of different CouncilDistrictCode")
plt.xlabel("CouncilDistrictCodes")
plt.ylabel("Count");

### YearBuiltRange - CouncilDistrictCode

In [ ]:
pd.Series(df.CouncilDistrictCode.unique()).sort_values()

In [ ]:
plt.figure(figsize=(12, 8))
g = sns.histplot(data=df.sort_values(by="CouncilDistrictCode"), x="YearBuiltRange", hue="CouncilDistrictCode", palette=sns.color_palette('bright', df.CouncilDistrictCode.unique().shape[0]), edgecolor=".2", multiple="stack")
for c in g.containers:
    labels = [v.get_height() if v.get_height() > 0 else '' for v in c]
    g.bar_label(c, labels=labels, label_type='center', c="white")
plt.title("Number of buildings built by discricts")
plt.xticks(rotation=45, horizontalalignment="right");

- Il semblerait que parmis nos données, le district 5 n'existait pas avant 1920
- À partir des années 1970, on commence à avoir une bonne répartition de construction de bâtiments dans tous les districts avec une légère préférence de construction dans le district 7

In [ ]:
next_feature()

## Neighborhood

In [ ]:
df.Neighborhood.isna().value_counts()

In [ ]:
df.Neighborhood.unique()

On remarque des petites erreurs de majuscules et d'informations rajoutées

In [ ]:
df.Neighborhood.replace(
    {
        "North": "NORTH",
        "Central": "CENTRAL",
        "Northwest": "NORTHWEST",
        "Delridge": "DELRIDGE",
        "Ballard": "BALLARD",
        "DELRIDGE NEIGHBORHOODS": "DELRIDGE",
    },
    inplace=True,
)
df.Neighborhood.unique()

In [ ]:
len(df.Neighborhood.unique())

### Neighborhood - Latitude/Longitude

Il y a forcément un lien important entre neighborhood et la position géographique du bâtiment dans la ville de Seattle.

On peut donc se questionner sur si avoir la Latitude/Longitude et une information sur Neighborhood est important

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(data=df, x="Longitude", y="Latitude", hue="Neighborhood")
plt.title("Neighborhoods by Longitude/Latitude");

Contrairement à quand on affichait les ZipCode on a cette fois-ci un résultat beaucoup plus propre.

On remarque bien les clusters qui se sont formés.

On peut quand même se poser la question de la pertienence de la feature quartier par rapport à la Latitude/Longitude. On pourra regarder ça plus en détails lorqu'on s'attaquera à nos targets

### Neighborhood - YearBuilt

In [ ]:
sns.heatmap(data=pd.crosstab(df.YearBuiltRange, df.Neighborhood))
plt.xticks(rotation=45, horizontalalignment="right");

- Dans les années 1900-1920, il y a eu beaucoup de bâtiments construits en centre ville
- Beaucoup de bâtiments construits autour des années 1960 dans le "GEATER SUWAMISH"

In [ ]:
next_feature()

## NumberofBuildings

In [ ]:
df.NumberofBuildings.isna().sum()

In [ ]:
(df.NumberofBuildings == 0).value_counts()

Il manque bien des données.

Un bâtiment ne peut pas avoir 0 bâtiments. Donc il y a un traitement à effectuer sur ces données.

In [ ]:
sns.countplot(data=df, x="NumberofBuildings")

Vu qu'il y a une majorité de valeurs à 1. On peut ne pas s'embêter avec un calcul de ce nombre pour les nulls et remplacer par 1.

In [ ]:
df.loc[df.NumberofBuildings.isna() | (df.NumberofBuildings == 0), "NumberofBuildings"] = 1

On peut se questionner sur si cette feature est importante ou non pour notre modèle. En effet, elle n'a également pas beaucoup de variance dans ses données

In [ ]:
df.NumberofBuildings.value_counts(normalize=True)

En suivant le principe de 90% de variance qu'on avais effectué au dessus. On peut donc supprimer cette feature.

In [ ]:
df.drop(columns="NumberofBuildings", inplace=True)

In [ ]:
next_feature()

## NumberofFloors

J'ai bien peur que cette feature soit similaire à NumberofBuildings.

Voici sa description : *"Number of floors reported in Portfolio Manager"*

In [ ]:
df.NumberofFloors.isna().sum()

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(data=df, x="NumberofFloors")

En réalité on a pas mal de variance à l'intérieur.

On souhaite donc corriger les éventuelles erreurs pour cette feature car elle pourrait nous intéresser. On remarque déjà qu'il y a des 0. On peut commencer par regarder si ces 0 sont normaux.

Je suis également intéressé de comprendre la feature de manière générale. En effet, comme il y a la feature NumberofBuildings on peut se demander si le nombre  d'étages saisis pour une ligne correspond à 
- La somme des étages de chaques bâtiments ?
- La moyenne d'étages pour chaque bâtiments ?
- Une autre méthode de calcul ?

In [ ]:
data.loc[data.NumberofBuildings > 20, ["NumberofBuildings","NumberofFloors"]]

Comme on a auparavant supprimé NumberofBuildings, je propose que l'on écarte également cette feature. Je n'ai pas l'impression que la définition de cette variable soit correctement définie (ex du 0 pour 111 bâtiments, c'est une université)

In [ ]:
df.drop(columns="NumberofFloors", inplace=True)
next_feature()

## PropertyGFATotal

Voici une feature très intéressante que l'on conservera à coup sûr pour nos prédictions. En effet on peut déjà dire que la surface joue un rôle très important dans la consommation électrique et les émission de CO².

In [ ]:
df.PropertyGFATotal.isna().sum()

In [ ]:
df.PropertyGFATotal.describe()

### PropertyGFATotal - YearBuiltRange

In [ ]:
plt.title("PropertyGFATotal per YearBuiltRange")
sns.scatterplot(data=df, x="YearBuiltRange", y="PropertyGFATotal")
plt.xticks(rotation=45, horizontalalignment="right");

On remarque un Outlier évident qu'on avait déjà pour NumberofFloors

In [ ]:
df[df.PropertyGFATotal > 8e6]

Je veux bien croire qu'une école consomme beaucoup. Mais déjà la valeur de 1900 pour l'année de construction est assez atypique, l'énergie utilisée de 8.7e7 me parait trop grande, la valeur de la surface également... Cette ligne a également un nombre de bâtiments énorme (d'après nos observations passées).

Je propose donc de supprimer cette ligne.

In [ ]:
df = df[df.PropertyGFATotal < 8e6]

In [ ]:
sns.scatterplot(data=df, x="YearBuiltRange", y="PropertyGFATotal")
plt.title("PropertyGFATotal per YearBuiltRange")
plt.xticks(rotation=45, horizontalalignment="right");

In [ ]:
df.groupby("YearBuiltRange").PropertyGFATotal.sum().reset_index()

In [ ]:
property_gfa_total_sum = df.groupby("YearBuiltRange").PropertyGFATotal.sum().reset_index()
sns.lineplot(data=property_gfa_total_sum, x="YearBuiltRange", y="PropertyGFATotal")
plt.title("Sum of PropertyGFATotals for each YearBuiltRange")
plt.xticks(rotation=45, horizontalalignment="right");

- De 1980 à maintenant, plus de surface a été créée
- Entre 1930 et 1950, pas beaucoup d'activité de création de surface
- Même si les données sont récupérées en pleine pleine intervalle 2010-2016, on remarque que moins de la moitiée de surface qu'avant a été créée (~0.75e7). Cela pourrait signifier une potentielle baisse pour l'intervalle 2010-2019.

### PropertyGFATotal - Latitude/Longitude

On peut afficher le poids sous forme de surface pour chaque bâtiment avec notre carte latitude/longitude

In [ ]:
# Neighborhood centers for latitude/longitude
neighborhood_centers = df.groupby("Neighborhood")[["Latitude", "Longitude"]].mean()

plt.figure(figsize=(18,12))
sns.scatterplot(df, x="Longitude", y="Latitude", size="PropertyGFATotal", sizes=(40, 1000), alpha=0.5, hue="Neighborhood")
for text in neighborhood_centers.index:
    latitude, longitude = neighborhood_centers.loc[text]
    plt.text(longitude, latitude, text, horizontalalignment='center')
plt.title("GFA sizes with latitude/longitude of building colored by neighborhood");

- Les plus grosses surfaces sont dans le centre (DOWNTOWN, EAST...)
- Les quartiers ont quasiement tous au moins un gros bâtiment

### PropertyGFATotal - PrimaryPropertyType

In [ ]:
property_gfa_total_agg = df.groupby("PrimaryPropertyType").PropertyGFATotal.agg(["mean", "min", "max"]).reset_index()
property_gfa_total_agg.head()

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(data=property_gfa_total_agg, x="PrimaryPropertyType", y="mean", label="Mean", color="green", linestyle="--")
sns.lineplot(data=property_gfa_total_agg, x="PrimaryPropertyType", y="min", label="Min", color="red", linestyle="--")
g = sns.lineplot(data=property_gfa_total_agg, x="PrimaryPropertyType", y="max", label="Max", color="red", linestyle="--")
lines = g.get_lines()
plt.fill_between(lines[0].get_xdata(), lines[1].get_ydata(), lines[2].get_ydata(), color="red", alpha=.1)
plt.ylabel("Min/Mean/Max PropertyGFATotal")
plt.title("PrimaryPropertyTypes for Min/Mean/Max PropertyGFATotal")
plt.xticks(rotation=45, horizontalalignment="right");

- Des valeurs max présentes pour les Hospital, Large Office, Other, University
- Valeurs moyennes hautes pour Hospital, LargeOffice
- Des minimums similaires proches des moyennes/max pour Office, Refrigerated Warehouse, Restaurant, Self-Storage Facility, Small/Mid-Sized Office, Worship Facility. Il y a donc peu de variance pour ces types de bâtiments.

In [ ]:
next_feature()

## PropertyGFAParking

J'ai personnelement du mal à comprendre s'il existerait un lien entre un parking et des émissions en CO2 ou consommation électrique. Peut être pour l'éclairage, les voitures qui y stationnent ou l'entretien.

In [ ]:
df.PropertyGFAParking.isna().sum()

In [ ]:
(df.PropertyGFAParking == 0).value_counts(normalize=True)

82% de valeurs égales à 0 ça fait beaucoup. Regardons la distribution des 17 autres %

In [ ]:
property_gfa_parking_non_0 = df.loc[df.PropertyGFAParking != 0]
sns.displot(data=property_gfa_parking_non_0, x="PropertyGFAParking")

Rappel : les valeurs proche de 0 ne sont pas égales à 0

- Courbure plutôt exponentielle des GFA du parking

### PropertyGFAParking - PropertyGFATotal

In [ ]:
sns.scatterplot(data=df, x="PropertyGFAParking", y="PropertyGFATotal")

- Tous les bâtiments n'ont pas forcément de parking (droite verticale en 0)
- Linéarité pas forcément évidente (hors droite verticale). Il y a quand même un corrélation entre la surface du parking et totale

In [ ]:
property_gfa_parking_non_0.PropertyGFATotal.corr(property_gfa_parking_non_0.PropertyGFAParking)

On peut faire un petit feature engineering pour rajouter s'il y a un parking ou non

In [ ]:
df["Parking"] = df.PropertyGFAParking > 0

### Parking - Latitude/Longitude

In [ ]:
plt.figure(figsize=(14,9))
sns.scatterplot(data=df, x="Longitude", y="Latitude", hue="Parking", size="PropertyGFAParking", sizes=(40,400), alpha=0.7)
plt.title("Parking & Parking sizes with Latitude/Longitude");

- Peu de Parkings dans le sud. S'il y en a il n'y a pas beaucoup de place
- Il y a des "clusters" de parking (notamment dans le nord)
- La côte est remplie de parking

In [ ]:
neighborhood_gfa_sum = df.groupby("Neighborhood").PropertyGFAParking.sum().reset_index()
neighborhood_gfa_sum.head()

In [ ]:
sns.lineplot(data=neighborhood_gfa_sum, x="Neighborhood", y="PropertyGFAParking")
plt.xticks(rotation=45, horizontalalignment="right");

En effet, certains quartiers étant plus fréquentés, on y retrouve une plus grande surface de parking

In [ ]:
next_feature()

## PropertyGFABuilding(s)

In [ ]:
df["PropertyGFABuilding(s)"].isna().sum()

In [ ]:
# Contrôle de la différence des GFAs
(df.PropertyGFAParking + df["PropertyGFABuilding(s)"] - df.PropertyGFATotal != 0).sum()

L'information du PropertyGFATotal étant comprise dans PropertyGFABuilding(s) et PropertyGFAParking. On a donc pas besoin du PropertyGFATotal

In [ ]:
df.drop(columns="PropertyGFATotal", inplace=True)

In [ ]:
(df["PropertyGFABuilding(s)"] != 0).value_counts()

On a uniquement des propertyGFABuildings différents de 0 ce qui semble cohérent

In [ ]:
sns.scatterplot(data=property_gfa_parking_non_0, x="PropertyGFAParking", y="PropertyGFABuilding(s)");
property_gfa_parking_non_0.PropertyGFAParking.corr(df["PropertyGFABuilding(s)"])

### Building GFA - Latitude/Longitude

In [ ]:
plt.figure(figsize=(14,9))
sns.scatterplot(data=df, x="Longitude", y="Latitude", size="PropertyGFABuilding(s)", sizes=(40,800), alpha=0.7, hue="Neighborhood")
plt.title("Building sizes with Latitude/Longitude and Neighborhood");

Même conclusion que pour l'analyse avec TotalGFA

In [ ]:
next_feature()

On utilisera les variables Primary, Secondary et Third PropertyUseTypes pour retrouver celle-ci

In [ ]:
df.drop(columns="ListOfAllPropertyUseTypes", inplace=True)
next_feature()

## LargestPropertyUseType

In [ ]:
df.LargestPropertyUseType.isna().sum()

In [ ]:
df[df.LargestPropertyUseType.isna()]

In [ ]:
df.loc[df.LargestPropertyUseType.isna(), "LargestPropertyUseType"] = df.loc[df.LargestPropertyUseType.isna(), "PrimaryPropertyType"]
df.LargestPropertyUseType.isna().value_counts()

In [ ]:
order = df.LargestPropertyUseType.value_counts().sort_values(ascending=False).index

plt.figure(figsize=(18,6))
sns.countplot(data=df, x="LargestPropertyUseType", order=order)
plt.title("LargestPropertyUseType count")
plt.xticks(rotation=45, horizontalalignment="right");

- Nombre de LargestPropertyUseType représenté par une fonction exponentielle
- Beaucoup de types avec peu de données

### LargestPropertyUseType - PropertyGFABuilding(s)

In [ ]:
plt.figure(figsize=(18,6))
sns.stripplot(data=df, x="LargestPropertyUseType", y="PropertyGFABuilding(s)", order=order, alpha=0.8)
sns.stripplot(data=df, x="LargestPropertyUseType", y="PropertyGFAParking", order=order, alpha=0.8)
plt.ylabel("GFA")
plt.xticks(rotation=45, horizontalalignment="right");

Je serais bien tenté de faire un feature engineering et de généraliser ces données en des catégories plus simples mais du fait de la variance importante dans certains groupes comme Office, College/University, Hospital... J'ai peur de perdre beaucoup trop d'information.

À la rigueur on pourrait regrouper certaines catégories similaires comme Restaurant, Social/Meeting Hall, Repair Services, Museum, Automobile Dealership... Ce qui nous permettrait certainement de réduire nos dimensions finales surtout si on utiliser un OneHotEncoder

À voir par la suite. Notamment après avoir analysé les 2 autres UseTypes.

In [ ]:
next_feature()

## LargestPropertyUseTypeGFA

In [ ]:
df.LargestPropertyUseTypeGFA.isna().sum()

In [ ]:
df[df.LargestPropertyUseTypeGFA.isna()]

PropertyGFABuilding(s) étant défini, on va remplacer les valeurs par celles de cette feature

In [ ]:
df.loc[df.LargestPropertyUseTypeGFA.isna(), "LargestPropertyUseTypeGFA"] = df.loc[df.LargestPropertyUseTypeGFA.isna(), "PropertyGFABuilding(s)"]
df.LargestPropertyUseTypeGFA.isna().sum()

In [ ]:
df.LargestPropertyUseTypeGFA.describe()

In [ ]:
sns.countplot(x=pd.cut(df.LargestPropertyUseTypeGFA, bins=np.append(np.arange(0, 5e5, 5e4), math.inf)))
plt.xticks(rotation=45, horizontalalignment="right");

- Courbure exponentielle.
- La majorité des données sont comprises entre 0 et 1.5e5

### LargestPropertyUseTypeGFA - LargestPropertyUseType

In [ ]:
plt.figure(figsize=(16,10))
sns.barplot(data=df, x="LargestPropertyUseType", y="LargestPropertyUseTypeGFA", order=order)
plt.xticks(rotation=45, horizontalalignment="right");

### LargestPropertyUseType - LargestPropertyUseTypeGFA

On peut comparer les GFA totaux entre eux. On doit comparer le GFA des parkings avec la donnée "Parking" de LargestPropertyUseType

In [ ]:
largest_property_type_gfas = df.groupby("LargestPropertyUseType").LargestPropertyUseTypeGFA.sum()
largest_property_type_gfas.head()

In [ ]:
parking_gfa = df.PropertyGFAParking.sum()
buildings_gfa = df["PropertyGFABuilding(s)"].sum()
print(parking_gfa, buildings_gfa)

In [ ]:
largest_parking_gfa = largest_property_type_gfas["Parking"]
largest_buildings_gfa = largest_property_type_gfas[largest_property_type_gfas[~largest_property_type_gfas.index.isin(["Parking"])].index].sum()
print(largest_parking_gfa / parking_gfa, largest_buildings_gfa / buildings_gfa)

- LargestPropertyUseType représente 15% du GFA total pour les parkings et 92% du GFA total pour les bâtiments

Il manque donc 85% du GFA des parkings que l'on espère retrouver dans le SecondaryPropertyUseType ou ThirdPropertyUseType

In [ ]:
plt.figure(figsize=(16,10))
sns.barplot(data=df, x="LargestPropertyUseType", y="LargestPropertyUseTypeGFA", order=order)
plt.xticks(rotation=45, horizontalalignment="right");

In [ ]:
next_feature()

In [ ]:
next_feature()

In [ ]:
next_feature()

In [ ]:
next_feature()

## PropertyUseTypes

On peut maintenant utiliser ces variable pour effectuer des comparaisons

In [ ]:
df.SecondLargestPropertyUseType.isna().value_counts(normalize=True)

In [ ]:
df.SecondLargestPropertyUseTypeGFA.isna().value_counts(normalize=True)

In [ ]:
df.ThirdLargestPropertyUseType.isna().value_counts(normalize=True)

In [ ]:
df.ThirdLargestPropertyUseTypeGFA.isna().value_counts(normalize=True)

Les données semble correctes. On peut se dire que pour un ThirdLargestPropertyUseType null, on a un ThirdLargestPropertyUseTypeGFA null.

- Le calcul de PrimaryPropertyType est défini comme : *"function that accounts for more than 50% of a property."*. Je ne pense pas qu'un tel calcul aide pour nos prédiction. Je propose donc de retirer cette feature. Les mêmes données seront présentes dans LargestPropertyUseType

In [ ]:
df.drop(columns="PrimaryPropertyType", inplace=True)

On peut commencer par regarder les PropertyUseTypes présents dans les autres features que LargestPropertyUseType

In [ ]:
largest_use_types = df.LargestPropertyUseType.unique()
second_use_types = df.SecondLargestPropertyUseType.unique()
third_use_types = df.ThirdLargestPropertyUseType.unique()
all_use_types = list((set(largest_use_types) | set(second_use_types) | set(third_use_types)))
all_use_types.remove(np.nan)
all_use_types[:5]

In [ ]:
set(all_use_types).difference(largest_use_types)

On remarque la valeur "Multifamily Housing". L'objectif est toujours d'effectuer des prédictions sur des bâtiments non résidentiels mais comme le bâtiment n'est pas principalement destiné à ça on peut le garder.

Par rapport à ce qui a été dit avant, regardons si les parking représente 85% des types restants.

In [ ]:
second_property_use_type_gfa = df.groupby("SecondLargestPropertyUseType").SecondLargestPropertyUseTypeGFA.sum()
second_property_use_type_gfa_parking = second_property_use_type_gfa["Parking"].sum()
third_property_use_type_gfa = df.groupby("ThirdLargestPropertyUseType").ThirdLargestPropertyUseTypeGFA.sum()
third_property_use_type_gfa_parking = third_property_use_type_gfa["Parking"].sum()

(second_property_use_type_gfa_parking + third_property_use_type_gfa_parking) / parking_gfa

Hmmm. Je m'attendais à avoir 0.85 mais j'ai 0.97. Est-ce une erreur de calcul ou une incohérence du dataset ? Je ne sais pas mais je vais quand même conserver cette analyse

In [ ]:
use_types_gfa = pd.DataFrame({"type":all_use_types, "gfa": np.zeros(len(all_use_types))}).set_index("type")
use_types_gfa.head()

In [ ]:
for property_type in all_use_types:
    if property_type in largest_property_type_gfas.index:
        use_types_gfa.loc[property_type]["gfa"] += largest_property_type_gfas[property_type]
    if property_type in second_property_use_type_gfa.index:
        use_types_gfa.loc[property_type]["gfa"] += second_property_use_type_gfa[property_type]
    if property_type in third_property_use_type_gfa.index:
        use_types_gfa.loc[property_type]["gfa"] += third_property_use_type_gfa[property_type]

use_types_gfa = use_types_gfa.reset_index()
use_types_gfa.head()

In [ ]:
df.columns

In [ ]:
total_gfa = df.PropertyGFAParking + df["PropertyGFABuilding(s)"]
total_gfa.head()

In [ ]:
plt.figure(figsize=(20,8))
sns.barplot(data=use_types_gfa, x="type", y="gfa")
plt.title("Total GFA per PropertyUseType")
plt.xticks(rotation=45, horizontalalignment="right");

On peut maintenant en tirer des conclusions

- Beaucoup de PropertyUseTypes ont un GFA très bas.
- Il y clairement beaucoup plus d'Office ce qui explique le gros pic.
- Le parking représente également une grande partie du GFA, ce qui peut expliquer pourquoi la feature est à part (PropertyGFAParking)

In [ ]:
next_feature()

Feature pas très importante. Cela pourrait influencé négativement les résultats de prédiction.

Ex: Si on a un bâtiment avec les mêmes données mais pas le même YearsENERGYSTARCertified

In [ ]:
df.YearsENERGYSTARCertified.isna().value_counts(normalize=True)

En plus on a 93% de non définis

In [ ]:
df.drop(columns="YearsENERGYSTARCertified", inplace=True)

In [ ]:
next_feature()

## ENERGYSTARScore

Définition :

*"Based on the information you enter about your building, such as its size, location, number of occupants, number of PCs, etc., the score’s algorithm estimates how much energy the building would use if it were the best performing, the worst performing, and every level in between. It then compares the actual energy data you entered to the estimate to determine where your building ranks relative to its peers."* [source](https://www.energystar.gov/buildings/benchmark/understand_metrics/how_score_calculated#:~:text=How%20the%20Calculation%20Works)

*"buildings with a score of 50 perform better than 50% of their peers"* [source](https://www.energystar.gov/buildings/tools-and-resources/portfolio-manager-technical-reference-energy-star-score)

*"The peer group for comparison is the national population of other buildings with the same primary business activity"* [source](https://portfoliomanager.energystar.gov/pdf/reference/ENERGY%20STAR%20Score.pdf)

In [ ]:
df.ENERGYSTARScore.isna().value_counts(normalize=True)

- 1/3 de valeurs définies pour l'ENERGYSTARScore

In [ ]:
valid_energy_score_df = df[df.ENERGYSTARScore.notna()]
plt.title("ENERGYSTARScore distribution")
sns.histplot(x=valid_energy_score_df.ENERGYSTARScore, kde=True);

In [ ]:
(valid_energy_score_df.ENERGYSTARScore > 50).sum() / valid_energy_score_df.shape[0]

- Beaucoup de grandes valeurs. D'après la définition cela signifie qu'on a, pour la portion de bâtiments dans la ville de Seattle, 74% des bâtiments au dessus de la moyenne national pour un même secteur d'activité

### ENERGYSTARScore - YearBuiltRange

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(data=pd.crosstab(pd.cut(valid_energy_score_df.ENERGYSTARScore, bins=np.arange(0,101,10)), valid_energy_score_df.YearBuiltRange))
plt.xticks(rotation=45, horizontalalignment="right");

- En 2010, très peu de bâtiment sont en dehors de l'intervalle 80-100.
- Entre 1950 et 1980, le plus de bâtiment construits avec un ENERGYSTARScore faible a été atteint.

In [ ]:
next_feature()

## SiteEnergyUse(kBtu)

SiteEnergyUse(kBtu) est une de nos targets.

In [ ]:
df["SiteEnergyUse(kBtu)"].isna().sum()

SiteEnergyUse(kBtu) étant notre target, on va juste supprimer la ligne pour cette valeur null

In [ ]:
df = df.loc[df["SiteEnergyUse(kBtu)"].notna()]
df.shape

In [ ]:
sns.histplot(df, x="SiteEnergyUse(kBtu)", kde=True)

In [ ]:
df["SiteEnergyUse(kBtu)"].skew()

On a clairement des données skewées sur la droite pour cette target, c'est à retenir pour quand on fera nos encodages. On pourra certainement utiliser le log qui est une très bonne méthode pour scaler des données skewée

### SiteEnergyUse(kBtu) - GFA

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(data=df, x="SiteEnergyUse(kBtu)", y=total_gfa, hue="Parking", palette=sns.color_palette("blend:blue,yellow", n_colors=2), alpha=0.7)
plt.title("SiteEnergyUse by the TotalGFA by Parking")
plt.ylabel("TotalGFA");

### SiteEnergyUse(kBtu) - YearBuiltRange

In [ ]:
from pandas.api.types import is_numeric_dtype

def continuous_discret_viz(df, x, y):
    labels = pd.Series(df[x].unique()).sort_values()
    fig, axs = plt.subplots(1, 2, figsize=(22,6))
    axs[0].set_title(f"{y} for each {x}s")
    g = sns.boxplot(data=df, x=x, y=y, showfliers=False, showmeans=True, ax=axs[0])
    if(not is_numeric_dtype(df[x])):
        g.set_xticklabels(labels, rotation=45, ha="right")
    axs[1].set_title(f"{x} count")
    g = sns.countplot(data=df, x=x, ax=axs[1], edgecolor="black")
    if(not is_numeric_dtype(df[x])):
        g.set_xticklabels(labels, rotation=45, ha="right")
    plt.show()

continuous_discret_viz(df, "YearBuiltRange", target_cols[1])

- On consomme plus d'électricité de nos jours pour un quasi même nombre de bâtiments

In [ ]:
next_feature()

À supprimer

In [ ]:
df.drop(columns="DefaultData", inplace=True)

In [ ]:
next_feature()

## ComplianceStatus

Description :

*"Whether a property has met energy benchmarking requirements for the current reporting year."*

Cela nous importe peu dans notre cas d'étude

In [ ]:
df.ComplianceStatus.unique()

Mais on remarque qu'il y a des valeurs un peu bizzares

In [ ]:
df.loc[df.ComplianceStatus == "Missing Data", "SiteEnergyUse(kBtu)"].max()

Pour "Missing Data" il semblerait qu'il manque la target SiteEnergyUse. On peut donc retirer ces lignes.

In [ ]:
df = df[~(df.ComplianceStatus == "Missing Data")]

In [ ]:
df.loc[df.ComplianceStatus == "Error - Correct Default Data","ENERGYSTARScore"].isna().sum()

Il semblerait que cette donnée fasse référence au fait que l'ENERGYSTARScore ne soit pas null. On peut donc conserver ces lignes. On peut par contre supprimer la feature qui ne sera pas utile.

In [ ]:
df.drop(columns="ComplianceStatus", inplace=True)
next_feature()

## Outlier

Outlier peut nous permettre de justement détecter les outliers. C'est rare qu'un dataset nous fournisse directement l'information comme quoi une donnée est un outlier. Il faut donc en profiter.

On peut commencer par regarder combien il y a de données annotées comme Outlier et si c'est bien le cas.

In [ ]:
df.Outlier.isna().value_counts()

D'après cette variable il y aurait 15 outliers

In [ ]:
df.Outlier.unique()

In [ ]:
outliers = ["High outlier", "Low outlier"]
df[df.Outlier.isin(outliers)]

Il semblerait que les outliers soient considérés ainsi par leur ENERGYSTARScore

In [ ]:
df[["PropertyGFABuilding(s)", "SiteEnergyUse(kBtu)", "TotalGHGEmissions", "ENERGYSTARScore"]].describe()

In [ ]:
df.Outlier.isin(outliers).value_counts()

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(data=df, x="ENERGYSTARScore", y="TotalGHGEmissions", hue=df.Outlier.isin(outliers), alpha=0.5, palette=sns.color_palette("blend:blue,red", n_colors=2))

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(data=df, x="PropertyGFABuilding(s)", y="SiteEnergyUse(kBtu)", hue=df.Outlier.isin(outliers), alpha=0.5, palette=sns.color_palette("blend:blue,red", n_colors=2))

Or on voir que ce ne sont pas vraiment les seuls outliers et qu'ils font parti d'autres clusters déjà existant et pas annoté comme "Outliers". Il ne faudait peut-être pas faire confiance à la feature Outlier. D'Autant plus que ces "outliers" sont également caractérisés par différents types pour les bâtiments (ce ne sont pas uniquement des "Office")

In [ ]:
df.drop(columns="Outlier", inplace=True)
next_feature()

## TotalGHGEmissions

Voici notre 2ème target

In [ ]:
df.TotalGHGEmissions.isna().sum()

In [ ]:
df["TotalGHGEmissions"].describe()

In [ ]:
df.loc[df["TotalGHGEmissions"] < 0, "TotalGHGEmissions"] = 0

In [ ]:
sns.histplot(df, x="TotalGHGEmissions", kde=True)

In [ ]:
df.TotalGHGEmissions.skew()

Cette target est également swewée

### TotalGHGEmissions - LargestPropertyUseTypeGFA

In [ ]:
plt.figure(figsize=(16,10))
sns.scatterplot(data=df, x="LargestPropertyUseTypeGFA", y="TotalGHGEmissions")
sns.lineplot(x=[0, 1.75e6], y=[0, 1.3e4], color="green")
sns.lineplot(x=[0, 1.75e6], y=[0, 1e3], color="green")
plt.title("TotalGHGEmissions by LargestPropertyUseTypeGFA");

- Beaucoup de valeurs basses pour les 2 features
- Quelques outliers, on remarque 2 tendances linéaires différentes

### TotalGHGEmissions - Longitude/Latitude

In [ ]:
# Neighborhood centers for latitude/longitude
neighborhood_centers = df.groupby("Neighborhood")[["Latitude", "Longitude"]].mean()

plt.figure(figsize=(18,12))
sns.scatterplot(df, x="Longitude", y="Latitude", size="TotalGHGEmissions", sizes=(40, 1000), alpha=0.5, hue="Neighborhood")
for text in neighborhood_centers.index:
    latitude, longitude = neighborhood_centers.loc[text]
    plt.text(longitude, latitude, text, horizontalalignment='center')
plt.title("TotalGHGEmissions with latitude/longitude of building colored by neighborhood");

- 1 grosse valeur dans NORTHEAST et CENTRAL
- EAST semble être une zone à forte tendance d'émission (zone industrielle ?)

### TotalGHGEmissions - CouncilDistrictCode

In [ ]:
continuous_discret_viz(df, "CouncilDistrictCode", target_cols[0])

- La médiane des différents CouncilDistrictCode sont très proches les unes des autres.
- Les districts 3 et 7 ont de grandes valeurs max, ce qui peut s'expliquer par leur nombre

In [ ]:
next_feature()

## GHGEmissionsIntensity

Je n'avais pas retiré cette variable au début de mon analyse. Donc j'ai toujours l'analyse que j'ai effectué dessus mais le code ne marchera pas car la feature a été retirée plus haut

In [ ]:
#df.GHGEmissionsIntensity.isna().sum()

In [ ]:
#df.corrwith(df.GHGEmissionsIntensity)

### GHGEmissionsIntensity - TotalGHGEmissions

In [ ]:
#sns.scatterplot(data=df, x="GHGEmissionsIntensity", y="TotalGHGEmissions")

### GHGEmissionsIntensity - Longitude/Latitude

In [ ]:
# plt.figure(figsize=(18,12))
# sns.scatterplot(df, x="Longitude", y="Latitude", size="GHGEmissionsIntensity", sizes=(40, 1000), alpha=0.5, hue="Neighborhood")
# for text in neighborhood_centers.index:
#     latitude, longitude = neighborhood_centers.loc[text]
#     plt.text(longitude, latitude, text, horizontalalignment='center')
# plt.title("TotalGHGEmissions with latitude/longitude of building colored by neighborhood");

- EAST est cohérent avec les emissions en CO2 totales.
- Intensité globalement répartie entre quartiers. Peu d'intensité dans le GREATER DUWAMISH

In [ ]:
#next_feature()

C'était notre dernière feature

## Title

In [ ]:
nulls(df)

## Analyse


In [ ]:
def corr(df):
    plt.figure(figsize=(14, 11))
    sns.heatmap(df.corr(), annot=True)

corr(df)

In [ ]:
for o in df.select_dtypes("object"):
    s = df[o].unique().size
    if s > 20:
        print(f"{o} contains {s} different values")
    else:
        plt.figure()
        df[o].value_counts().plot.pie()

In [ ]:
df = df.drop(columns="YearBuilt")

## Utilité de l'ENERGYSTARTScore

Afin d'avoir une comparaison juste on peut utiliser la consommation pour une surface.

In [ ]:
[x for x in df.columns if "GFA" in x]

In [ ]:
emission_ft = df.TotalGHGEmissions / (df["PropertyGFABuilding(s)"] + df.PropertyGFAParking)
emission_ft.head()

In [ ]:
electricity_ft = df["SiteEnergyUse(kBtu)"] / (df["PropertyGFABuilding(s)"] + df.PropertyGFAParking)
electricity_ft.head()

In [ ]:
energy_range = pd.cut(df.ENERGYSTARScore, bins=range(0,101, 20))
energy_range.head()

In [ ]:
plt.figure(figsize=(14,8))
sns.scatterplot(x=emission_ft, y=electricity_ft, hue=energy_range, palette="RdBu", alpha=0.9)
plt.title("Electricity/ft by the Emission/ft colored by ENERGYSTARScore")
plt.xlabel("Emission/ft")
plt.ylabel("Electricity/ft")
plt.show()

Certains outliers sont présents et suggèrent qu'il y a une forte consommation d'électricité / emission (plus en haut à droite, plus il y a une consommation élevée pour la surface)

Ces résultats sont assez perturbant. En effet, on s'attends à ce que les meilleurs ENERGYSTARScore soient présents en bas à gauche du graphique. Ce n'est pas le cas. On remarque une distribution assez hétérogène des datapoints.

On peut émettre 2 hypothèses :

- L'ENERGYSTARScore n'est pas un score fiable
- L'ENERGYSTARScore ne prends pas uniquement en compte les émissions / consommation électrique (c'est un fait) et d'autres facteurs peuvent influencer grandement le score attribué à certains bâtiments. Il est notamment mentionné que dans le calcul de ce score le nombre de personnes ou encore l'environnement (climatique...).

# Export

On va sélectionner les variables importantes pour nos prédictions uniquement

In [ ]:
df.columns

In [ ]:
important = ['BuildingType', 'ZipCode', 'CouncilDistrictCode', 'Neighborhood',
        'Latitude', 'Longitude', 'PropertyGFAParking', 'PropertyGFABuilding(s)',
        'LargestPropertyUseType', 'LargestPropertyUseTypeGFA',
        'SecondLargestPropertyUseType', 'SecondLargestPropertyUseTypeGFA',
        'ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseTypeGFA',
        'SiteEnergyUse(kBtu)', 'TotalGHGEmissions', 'YearBuiltRange',
        'Parking', 'ENERGYSTARScore']
df = df[important]

In [ ]:
plt.figure(figsize=(9,7))
sns.heatmap(df.corr())

In [ ]:
df.columns

Certaines variables sont fortement corrélées et certaines pourraient être retirées. On fera un choix en fonction de la précision de nos prédictions

In [ ]:
df.to_csv("cleaned.csv", index=False)